# U.T.6 Redes Neuronales.
## Ajuste de la red
### Introducción
Las redes neuronales que pueden ser sustituidas por algoritmos de regresión o clasificación no suelen
ser muy profundas (muchas capas) por lo que no presentan graves problemas.

Es cuando avanzamos en el entrenamiento de redes más grandes cuando aparecen ciertos problemas:
- Desajustes en los crecimientos de los gradientes
- No tengamos acceso a suficientes datos
- El entrenamiento puede que nos lleve mucho tiempo
- Se puede sobreajustar a los datos

Para prevenir que un modelo aprenda patrones irrelevantes, la mejor aproximación es conseguir más
datos para entrenarlo. Si no es posible, reduciendo la capacidad de la red podremos forzar a que se
centre en los patrones relevantes

In [ ]:
Reducir el tamaño de la red (reducir capas o neuronas)
El verdadero problema en las redes neuronales es la generalización
model = models.Sequential()
model.add(layers.Dense(16, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model = models.Sequential()
model.add(layers.Dense(4, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(4, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid’))

model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

![](img/ut06_15.png)
### Desajustes en los gradientes.
El algoritmo de backpropagacion va desde la capa de salida y va subiendo por la estructura hasta la capa de entrada,
aplicando el reajuste de pesos de abajo a arriba

Problemas
- El gradiente que se va calculando al aplicarlo en este orden hace que las capas más cercanas a la entrada queden
prácticamente invariadas con lo que la solución nunca converge (desvanecimiento del gradiente)
- Los cambios se hace cada vez mayor, siendo enorme en las capas más cercanas a la entrada (estallido del gradiente)
haciendo que el problema tampoco converja
- En las NN de gran tamaño cada capa aprenderá a ritmos muy diferentes

#### Solución 1: Inicialización Glorot y He
inicializar los pesos de forma aleatoria. Keras usa la inicialización Glorot

keras.layers.Dense(10, activation="relu",  kernel_initializer="he_normal")
keras.layers.Dense(10, activation="relu", kernel_initializer="lecun_normal")


Restricciones
	Inicialización	Función de activación

	Glorot		None, tanh, logistic, softmax

	He		    ReLU and variants

	LeCun		SELU

#### Solución 2: Funciones que no saturan
- RELU. Buena aproximación pero algunas neuronas no aportarán nada
- Leaky RELU. Se puede controlar el ajuste a través del parámetro alfa. Valores típicos son 0.01, 0.1 o 0.2
- ELU. Mejora los tiempos de aprendizaje, tiene un parámetro alfa que se suele ajustar a 1. Tiene un gran coste
computacional
- SELU. Variante escalada de SELU. Se autonormaliza. Deben darse las siguientes condiciones: Entradas estandarizadas,
la capa oculta inicializada con LeCum, la arquitectura debe ser secuencial y todas las capas densas.

**SELU > ELU > Leake RELU >> RELU > Tang > Logistic**

#### Solución 3: Normalización Batch
Se basa en centrar en cero las entradas y normalizar, escalarlas y desplazarlas utilizando nuevos parámetros por
cada capa.

Si se añade como primera capa después de la entrada o del aplanado (Flatter) conseguiremos que se normalicen
las entradas y no será necesario que lo hagamos nosotros antes.

Se puede hacer justo antes de cada capa oculta (aunque hay estudios que defienden que se haga después).

Puede mejorar hasta grados del 2% solo añadiendo las capas sin afinar nada.

La desventaja es que añade complejidad al modelo, aumentando el tiempo de procesamiento, pero a cambio disminuye el
número de épocas para converger.

**Parámetros**
momentum. Este valor suele estar muy cercano a uno 0.9 y se añadirán más nuevos al valor cuanto más pequeño es el
tamaño del mini-batch


In [ ]:
model = keras.models.Sequential([
	keras.layers.Flatten(input_shape=[28, 28]),
	keras.layers.BatchNormalization(),
	keras.layers.Dense(300, activation="elu",
		kernel_initializer="he_normal"),
	keras.layers.BatchNormalization(),
	keras.layers.Dense(100, activation="elu",
		kernel_initializer="he_normal"),
	keras.layers.BatchNormalization(),
	keras.layers.Dense(10, activation="softmax")
])

keras.layers.Flatten(input_shape=[28, 28]),

Evita el reshape antes de la entrada de la red hecho hasta ahora

#### Solución 4: Recorte del gradiente
Se basa en recortar el tamaño del gradiente durante la backpropagación, evitando el problema de la explosión del
gradiente. Con esta técnica se limita el valor máximo que puede tener el cambio

El problema es que al realizar el corte se pueden presentar otros problemas, como que no se preserve la orientación
o se elimine el valor de una de las direcciones. Así un vector vale (0.9,100) y se aplica el recorte quedará en
(0.9,1) cambiando la dirección del vector. Del mismo modo si e hace un corte normalizado el resultado será (0.008, 1)
 con lo que la primera componente prácticamente se anula.

optimizer = keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="mse", optimizer=optimizer)

### Uso de redes entrenadas
No es una buena idea entrenar una red grande si existe otra que haga un trabajo similar. El entrenamiento puede ser
una tarea muy costosa y si podemos reutilizar partes de otros modelos ya entrenados y validados se adelantará mucho,
esta técnica se llama transferencia del aprendizaje

La idea principal es transferir las capas más cercanas a la entrada a nuestro modelo manteniendo los pesos. A
continuación, añadimos las capas superiores que creamos convenientes y entrenamos el modelo con nuestros datos,
forzando al cambio a las capas que hemos añadido

![](img/ut06_16.png)

Este procedimiento es muy válido y se utiliza mucho, lo único que deberemos tener algunas consideraciones:
- Si los modelos no tienen las mismas características de entrada, habrá que realizar un proceso de adaptación de
nuestros ejemplos al nuevo número de entradas.
- Generalmente la capa de salida debe ser remplazada para que se adecue al problema que estamos resolviendo.

Si no se ajusta, iremos liberando capas fijas de arriba a abajo de pocas en pocas y entrenando de nuevo para que
mejore el ajuste, cuantos más ejemplos de entrada tengamos más capas podremos liberar. No nos podemos olvidar ajustar
la tasa de aprendizaje correctamente, generalmente a un valor bajo.

Tenemos que tener especial cuidado con los estudios que se hacen en DeepLearning, en los que se muestran algoritmos
que funcionan muy bien con ciertos parámetros, o ciertos ajustes que son los que hay que usar. Estos estudios muestran
los resultados, pero muy pocas veces dicen los intentos que se han tenido que hacer hasta encontrar esos resultados,
por lo que un valor presentado parece que es el mejor pero lo que pasa es que los fallos no se muestran, por lo que no
se puede evaluar correctamente el funcionamiento.

No hay que obviar que esta técnica no funciona bien con pequeñas redes densas, mejora mucho con redes convolucionales profundas

Se verá más adelante con redes convolucionales

### Optimizadores rápidos
Entrenar DNN puede ser computacionalmente muy costoso y llevar una gran cantidad de tiempo. Para acelerar el proceso
se utilizan algoritmos de optimización rápidos.

#### Optimización del momento
Inicio lento de la optimización y en cada iteración aplicar más cantidad, el momento se toma como un valor de
aceleración a la velocidad actual de aprendizaje, en vez de la velocidad en sí misma. El valor general de este
optimizador es de 0.9.

La pregunta es para qué esta optimización. La respuesta es sencilla, las funciones pueden tener mínimos locales en
los que queden estancados los algoritmos, y esta “aceleración” extrá permitirá saltarlos


optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9)

#### NAG (Nesterov accelerated Gradient).
Es una pequeña variación del anterior que hace que sea más rápido que el GD o SGD.

optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9, nesterov=True)

#### AdaGrad
Escala la dirección del vector gradiente a en la dirección más empinada. Funciona muy bien para problemas cuadráticos,
pero se para muy pronto para NN. No se debe utilizar en problemas de Deep Learning ya que presenta el riesgo que pare
demasiado pronto en un mínimo local en vez del global.

#### RMSProp.
Este algoritmo mejora los problemas del anterior introduciendo un nuevo hiperparámentro beta que generalmente se
establece a 0.9.

optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)

#### ADAM.
Combina las ideas de momento y RMSPROP en un único algoritmo, por lo que incorpora dos nuevos hiperparámetros pero
generalmente los valores mostrados dan buenos resultados. (NADAM. Es similar a ADAM con una nueva optimización, que
generalmente hace que converja algo más rápido)


optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

#### Uso
Una vez definido y establecido el optimizador se pasará como parámetro a la función compile.

optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9)
model.compile(loss="mse", optimizer=optimizer)

![](img/ut06_17.png)

### Gestión de la tasa de aprendizaje
#### Introducción
Hasta ahora la tasa de aprendizaje permanece constante entre las diferentes épocas. Vamos a explorar la idea de que
esta varía dependiendo del momento de aprendizaje

#### La primera idea
Podemos utilizar para encontrar la tasa adecuada de aprendizaje es entrenar nuestro modelo con solo unos cientos de
épocas, incrementando de forma exponencial en cada una de ellas el valor de la tasa de aprendizaje. Una vez realizado
e muestra la curva de aprendizaje utilizando como tasa de aprendizaje un poco menor al punto en el que la curva vuelve
a crecer. Con esta tasa entrenar de nuevo el modelo completamente.

![](img/ut06_18.png)

#### La segunda idea
Empezar con una tasa de aprendizaje elevada y la vamos reduciendo en cada época de forma progresiva

##### Power scheduling.
Primero avanza muy rápido y después cada vez más lento, tiene un par de hipeparámetros a ajustar.

optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-4)

##### Exponential scheduling.
El descenso se hace de forma exponencial en un factor de diez cada vez.

def exponential_decay_fn(epoch):
	return 0.01 * 0.1**(epoch / 20)

lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
history = model.fit(X_train_scaled, y_train, [...], callbacks=[lr_scheduler])

##### Piecewise constant scheduling.
Divide el entrenamiento en tres periodos en función de las épocas y utiliza para cada una de ellas una tasa de
aprendizaje fija, pero diferente entre ellas.


def piecewise_constant_fn(epoch):
	if epoch < 5:
		return 0.01
	elif epoch < 15:
		return 0.005
	else:
		return 0.001
lr_scheduler = keras.callbacks.LearningRateScheduler(piecewise_constant_fn)
history = model.fit(X_train_scaled, y_train, [...],  callbacks=[lr_scheduler])

##### Performance scheduling.
Mide el error de validación cada N pasos y reduce la tasa de aprendizaje en un factor (hiperparámetro) cuando el error
deja cambiar.

lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

##### One cicle scheduling.
Divide el aprendizaje en dos periodos, en el primero la tasa de aprendizaje se incrementará y en el segundo descenderá
hasta alcanzar la inicial (hará una especie de pico) La elección de los valores inicial y pico se usa la siguiente
técnica. Para el pico se utilizará la técnica explicada en el primer párrafo, para el inicial se elegirá uno que sea
unas diez veces menor que el pico.

### Regularización
#### Introducción
Tal y como vimos en los capítulos primeros se pueden usar la regularización L2 (keras.regularizers.l2) y L1
(keras.regularizers.l1) para las NN o ambas a la vez (keras.regularizers.l1_l2). Hay que recordar que la penalización
se utilizará solo en la fase de entrenamiento.

layer = keras.layers.Dense(100, activation="elu, kernel_initializer="he_normal",
kernel_regularizer=keras.regularizers.l2(0.01))

![](img/ut06_19.png)

#### Dropout
La idea es ignorar ciertas neuronas en este ciclo de aprendizaje. El porcentaje de neuronas es un hiperpámetro a
ajustar, pero en NN suele estar entre 10% y 50%, más cerca del 20% en redes neuronales recurrentes y cerca del 40% o
50% en las convolucionales

Si observamos que el modelo se está sobreajustando podemos intentar incrementar el porcentaje de dropout, en caso de
subajuste se tendrá que bajar o incluso eliminar si así fuera necesario.

También es un factor el tamaño de la capa, en capas con muchas neuronas admiten mayor porcentaje de dropout que las
capas con pocas neuronas

model = keras.models.Sequential([
keras.layers.Flatten(input_shape=[28, 28]),
keras.layers.Dropout(rate=0.2),
keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
keras.layers.Dropout(rate=0.2),
keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
keras.layers.Dropout(rate=0.2),
keras.layers.Dense(10, activation="softmax")
])

![](img/ut06_21.png)

#### Regularización Max-Norm
Para cada neurona constriñe el valor de los pesos dentro de un radio (r) de tal manera que, al reducir, incrementa
la cantidad de regularización y mejorar el sobreajuste.

keras.layers.Dense(100, activation="elu",
		kernel_initializer="he_normal",
		kernel_constraint=keras.constraints.max_norm(1.))

#### Resumen
Si se necesita un modelo disperso se tiene que utilizar la regularización L1.

Si necesitamos un modelo de baja latencia (hace predicciones muy rápido) habrá que utilizar pocas capas y añadir
Batch Normalization antes de cada capa y posiblemente usar una función de activación muy rápida como Leaky RELU o RELU.

Si estamos haciendo un modelo de alto riesgo, Dropout mejorará el rendimiento.

![](img/ut06_22.png)

![](img/ut06_23.png)
